<a href="https://colab.research.google.com/github/nthanhkhang/vietnamese-pos-tagging/blob/main/Vietnamese_POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1500]:
!pip install pyvi
import string
import numpy
import sklearn.svm as svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import io
from collections import defaultdict
import random
from pyvi import ViTokenizer, ViPosTagger

In [1501]:
!wget https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_train.txt
!wget https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_test.txt

--2021-03-01 04:00:07--  https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1664500 (1.6M) [text/plain]
Saving to: ‘vi_train.txt.1’

vi_train.txt.1      100%[===================>]   1.59M  --.-KB/s    in 0.06s   

2021-03-01 04:00:07 (26.8 MB/s) - ‘vi_train.txt.1’ saved [1664500/1664500]

--2021-03-01 04:00:07--  https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

In [1502]:
tagsetDict = {"N" : 1,
              "Np" : 2,
              "Nc" : 3,
              "Nu" : 4,
              "V" : 5,
              "A" : 6,
              "P" : 7,
              "L" : 8,
              "M" : 9,
              "R" : 10,
              "E" : 11,
              "C" : 12,
              "I" : 13,
              "T" : 14,
              "B" : 15,
              "Y" : 16,
              "S" : 17,
              "X" : 19 }
inverseTagsetDict = {tagsetDict[k]:k for k in tagsetDict}

In [1503]:
wordBank = defaultdict()
bigramBank = defaultdict()
bigramDict = defaultdict(int)

bigramFreq = {x:[0]*18 for x in tagsetDict}
mostCommonBigrams = defaultdict()

prevParts = None

f_tagged_train = io.open("vi_train.txt", encoding='utf-8').readlines()
f_tagged_test = io.open("vi_test.txt", encoding='utf-8').readlines()

train = f_tagged_train[:10400]
test = f_tagged_test[:10000]
print(train[:10])
print(test[:10])

['Trên/E đường/N đi/V ,/, có/V một/M lần/N xe/N cô/N suýt/R rơi/V xuống/R vực/N ở/E đèo/N Ngoạn_Mục/Np ./.\n', '\n', 'Trong/E một/M trận/N đánh/V ác_liệt/A bên/N thành/N cổ/N Quảng_Trị/Np ,/, một/M loạt/N đạn/N pháo/N của/E kẻ_thù/N đã/R rơi/V trúng/A chỗ/N chiến_sĩ/N thông_tin/N Nguyễn_Văn_Thạc/Np .../... Hôm/N đó/P ngày/N 30/M -/- 7/M -/- 1972/M ./.\n', '\n', 'Sau/E khi/N H./Ny “/“ AK/Ny ”/” bị/V xộ/V khám/V ,/, Hoàng/Np đã/R hoàn_lương/V ,/, bây_giờ/P đang/R phụ/V việc/N cho/E gia_đình/N Quân/Np ./.\n', '\n', 'Hà/Np ,/, 21/M tuổi/N -/- làm/V nghề/N hớt_tóc/V ,/, khai/V :/: “/“ Ban_đầu/N nghe/V mấy/L ảnh/N nói/V thuốc/N lắc/V không/R nghiện/V nên/C em/N uống/V thử/V ./.\n', '\n', 'Theo/E anh/Nc Thông/Np ,/, người/N già/A ở/E Đức/Np thường/R sống/V một_mình/Ny và/C sống/V với/E những/L kỷ_vật/N của/E riêng/A mình/P mang/V về/R từ/E những/L chuyến/N du_lịch/V hay/C những/L tặng_vật/N của/E cả/P một/M đời/N ./.\n', '\n']
['Trên/E đường/N xuất_hiện/V nhiều/A cặp/N cha/N -/- con/N ,/, mẹ/

In [1504]:
for line in train:
    l_split = line.split()
    for i,w in enumerate(l_split):
        parts = w.split("/")
        if i >= 1:
            prevParts = l_split[i-1].split("/")
        if len(parts) == 1 or \
            parts[1] not in tagsetDict:
            continue

        if i >= 2 and prevParts[1] in string.punctuation:
            prevParts = l_split[i-1].split("/")

        word = parts[0]
        pos = parts[1]

        if i >= 1:
            prevWord = prevParts[0]
            prevPos = prevParts[1]
            bigramBank[ word ] = (pos, prevWord, prevPos)
            bigramDict[ (pos, prevPos) ] += 1

        if word not in wordBank:
            wordBank[word] = [pos]
        else:
            wordBank[word] += [pos]
#print(wordBank)

In [1505]:
bi_grams=[]
tri_grams=[]
qua_grams=[]
for sentence in set(wordBank):
  temp=0
  for s in list(sentence):
    if s == "_":
      temp+=1
    if temp == 0:
      bi_grams.append(sentence)
    elif temp == 2:
      tri_grams.append(sentence)
    elif temp == 4:
      qua_grams.append(sentence)
#print(bi_grams)
#print(tri_grams)
#print(qua_grams)

In [1506]:
#finished getting training data
for k in bigramFreq:
    maxFreq = 0
    maxPos = "X"
    for i,x in enumerate(bigramFreq[k]):
        if x > maxFreq:
            maxFreq = x
            maxPos = inverseTagsetDict[i]
    mostCommonBigrams[k] = maxPos

In [1507]:
def feature(word,wordIdx, lineSize, line):
    feat = [1]
    sentPercent = float(wordIdx)/float(lineSize)
    feat.append(sentPercent)

    if word[0].isupper() and wordIdx != 0:
        feat.append(1)
    else:
        feat.append(0)

    posIdx_array = ([0] * len(tagsetDict))
    posSet = []
    if word in wordBank:
        posSet = wordBank[word]
    else:
        if wordIdx == 0:
            posSet = list(tagsetDict.keys())[0] 
            posIdx_array[tagsetDict[posSet]] = 1
            return feat + posIdx_array + [0]
        else:
            prevWord = line[wordIdx-1]
            if prevWord in wordBank:
                prevPos = wordBank[prevWord]
                maxPos = mostCommonBigrams[prevPos]
                if prevPos == "E":
                    maxPos = 3
                posIdx_array[tagsetDict[maxPos]] = 1
                feat += posIdx_array + [tagsetDict[maxPos]]
                return feat
            else:
                posSet = list(tagsetDict.keys())[0]  
                posIdx_array[tagsetDict[posSet]] = 1
                feat += posIdx_array + [0]
                return feat

    for pos in posSet:
        posIdx = tagsetDict[pos]
        posIdx_array[posIdx] += 1.0/len(wordBank[word])
    feat += (posIdx_array) + [0]
    return feat

In [1508]:
#finished creating y and xtrain
y = []
X_train = []
for line in train:
    l_split = line.split()
    for i,w in enumerate(l_split):
        parts = w.split("/")
        word = parts[0]
        len_line = len(l_split)

        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue
        y.append(wordBank[word][0])
        X_train.append(feature(word,i,len_line, l_split))

In [1509]:
print (len(X_train))
print (len(y))
print(X_train[0:10])
print(y[0:10])
train_fit = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train,y)

99383
99383
[[1, 0.0, 0, 0, 0, 0, 0, 0, 0, 0.0625, 0, 0, 0, 0, 0.9375, 0, 0, 0, 0, 0, 0, 0], [1, 0.058823529411764705, 0, 0, 1.0000000000000018, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.11764705882352941, 0, 0, 0, 0, 0, 0, 0.900326797385618, 0, 0, 0, 0, 0.0931372549019607, 0, 0, 0, 0.006535947712418301, 0, 0, 0, 0], [1, 0.23529411764705882, 0, 0, 0, 0, 0, 0, 0.9912536443148805, 0, 0, 0, 0, 0.0058309037900874635, 0, 0, 0, 0.0029154518950437317, 0, 0, 0, 0], [1, 0.29411764705882354, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0000000000000189, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.35294117647058826, 0, 0, 0.974358974358972, 0, 0, 0, 0.02564102564102564, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.4117647058823529, 0, 0, 0.990566037735849, 0, 0, 0, 0.009433962264150943, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.47058823529411764, 0, 0, 0.7162162162162147, 0, 0.28378378378378405, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.5294117647058824, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [1516]:
#finished getting testing data
X_test = []
correct_results = []
for line in test:
    l_split = line.split()
    for i,w in enumerate(l_split):
        len_line = len(l_split)
        parts = w.split("/")
        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue
        word = parts[0]
        pos = parts[1]
        if word in string.punctuation or word == ":.":
            continue
        X_test.append(feature(word,i,len_line,l_split))
        correct_results.append(pos)
#print(X_test)

In [1517]:
predicted_results = train_fit.predict(X_test)
print("Predicted Results:",predicted_results[:12])
print("Correct Results  :",correct_results[:12])

numCorrect = 0
for cor,pred in zip(correct_results, predicted_results):
    if cor == pred:
        numCorrect += 1

accuracy = 1.0*numCorrect/len(correct_results)
print("Accuracy:",accuracy*100)

Predicted Results: ['E' 'N' 'V' 'A' 'N' 'N' 'N' 'N' 'N' 'C' 'V' 'P']
Correct Results  : ['E', 'N', 'V', 'A', 'N', 'N', 'N', 'N', 'N', 'C', 'C', 'T']
Accuracy: 87.02016583068891


In [1512]:
import unicodedata as ud
import re
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [1513]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in tri_grams:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [1514]:
def toString(wl):
  wl=longest_matching(wl, bi_grams, tri_grams)
  X=[]
  A=[]
  text=""
  for i in set(wl):
    if i ==".":
      A="./."
    else:
      X.append(feature(i,1,1,1))
      A=str(train_fit.predict(X))
  #print(A)  
  for i in range(len(wl)):
    text+=wl[i]
    text+='/'
    text+=str(A[2])
    text+=' '
  return text

In [1515]:
if __name__ == "__main__":
    wr ="Dù khá đắt nhưng tôi vẫn đồng ý."
    wl = ViTokenizer.tokenize(wr)
    print(wr)
    wl = wl.split()  
    for i in wl:
      print(toString(i),end='')

Dù khá đắt nhưng tôi vẫn đồng ý.
Dù/C khá/R đắt/A nhưng/C tôi/P vẫn/R đồng_ý/V ./. 